In [2]:
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from pyclim_noresm.general_util_funcs import global_avg
from workflow.scripts.utils import (calc_error, calc_abs_change,
                                compute_annual_emission_budget) 
                                    
import matplotlib
import numpy as np
import yaml
import dataframe_image as dfi
from functools import partial
from scipy.stats import ttest_ind, t

In [3]:
keys = list(snakemake.input.keys())
timeslice = snakemake.params.get('time_slice', slice(3,-1))

In [4]:
exp_keys = sorted([k for k in keys if 'exp' in k.split('_')])
ctrl_keys = sorted([k for k in keys if 'ctrl' in k.split('_')])
remainingkeys = list(set(keys)-set(exp_keys+ctrl_keys+ ['areacello']))
areapaths = {s.split('_')[2].split('.')[0] : s for s in snakemake.input['areacello']}

In [5]:
data={}
error={}
for exp_key, ctrl_key in zip(exp_keys,ctrl_keys):
    exp_paths = sorted(snakemake.input[exp_key])
    ctrl_paths = sorted(snakemake.input[ctrl_key])
    vname = xr.open_dataset(exp_paths[0]).variable_id
    models = [pst.split('_')[-2].split('.')[0] for pst in exp_paths]
    if 'emidust' in exp_key.split('_') or 'concdust' in exp_key.split('_'):
        iterator = zip(exp_paths, ctrl_paths, models)
    else:
        iterator = zip(exp_paths, ctrl_paths)
    data[vname] = {}
    error[vname] = {}
    for paths in iterator:
        if 'emidust' in exp_key.split('_') or 'concdust' in exp_key.split('_'):
            if snakemake.config['regrid_params']:
                ga = xr.open_dataset('workflow/input_data/common_grid.nc')
            else:
                ga = xr.open_dataset(areapaths[paths[2]])
            ds_ctrl = xr.open_dataset(paths[1])
            ds_exp = xr.open_dataset(paths[0])
            if ds_ctrl.cf.bounds.get('time'):
                ds_ctrl = ds_ctrl.drop(ds_ctrl.cf.bounds.get('time'))
                
            if ds_exp.cf.bounds.get('time'):
                ds_exp = ds_exp.drop(ds_exp.cf.bounds.get('time'))
            exp_budget, exp_std = compute_annual_emission_budget(ds_exp, ga)
            ctrl_budget, ctrl_std = compute_annual_emission_budget(ds_ctrl,ga)
            delta_emi = exp_budget-ctrl_budget
            data[vname][ds_ctrl.source_id] = float(delta_emi.values)
            sdterror = ((exp_std+ctrl_std)/2).values
            tval = data[vname][ds_ctrl.source_id]/(sdterror*np.sqrt(2/len(ds_ctrl.time)))
            p = t.sf(np.abs(tval),len(ds_ctrl.time))
            error[vname][ds_ctrl.source_id] = p
        else:
            ds_exp = xr.open_dataset(paths[0]).load()
            ds_ctrl = xr.open_dataset(paths[1]).load()
            if ds_ctrl.cf.bounds.get('time'):
                ds_ctrl = ds_ctrl.drop(ds_ctrl.cf.bounds.get('time'))
                
            if ds_exp.cf.bounds.get('time'):
                ds_exp = ds_exp.drop(ds_exp.cf.bounds.get('time'))
            ds_ctrl=ds_ctrl.reset_coords()
            ctrlM = global_avg(ds_ctrl.isel(time=timeslice))
            expM = global_avg(ds_exp.isel(time=timeslice))
            diff = expM[vname].mean()-ctrlM[vname].mean()
            tval,p = ttest_ind(expM[vname],ctrlM[vname])
            error[vname][ds_ctrl.source_id] = p
            data[vname][ds_ctrl.source_id] = diff 

In [6]:
for key in remainingkeys:
    paths = snakemake.input[key]
    if isinstance(paths, str):
        paths = [paths]
    for path in paths:
        if path.endswith('yaml'):
            
            with open(path,'r') as f:
                ds = yaml.safe_load(f)
            if ds.get('name'):
                source_id = ds.pop('name')
            for subk in ds:
                vname = "{}_{}".format(key,subk)
                if subk in ['delta_ems'] or vname in ['feedback_Direct_dCdT','feedback_Direct_dT_dEms', 'feedback_Clouds_dCdT','feedback_Clouds_dT_dEms	']:
                    continue
                else:
                    if vname not in data.keys():
                        data[vname] = {}
                        error[vname] = {}

                    data[vname][source_id] = np.mean(ds[subk]['data'])
                    error[vname][source_id] = float(np.std(ds[subk]['data']))
#             else:
#                 data = {}
#                 error = {}
#                 for subk in ds:
#                     data[subk]
        else:
            ds = xr.open_dataset(path)
            vname = list(ds.data_vars)[0]
            source_id = ds.source_id
            if vname not in data.keys():
                data[vname] = {}
                error[vname] = {}
            if 'year' in ds.dims:
                ds = ds.rename_dims(year='time')
            erf = global_avg(ds)
            
            data[vname][source_id] = float(erf[vname].mean(dim='time').values)
            tval = data[vname][source_id]/(float(erf[vname].std(dim='time').values)/np.sqrt(len(erf.time)))
            p = t.sf(np.abs(tval),len(erf.time))
            error[vname][f'{source_id}'] = p

In [7]:
df = pd.DataFrame(data, dtype=np.float)

dfe = pd.DataFrame(error, index=df.index,columns=df.columns, dtype=np.float)

df['ERF/AOD'] = df['ERFt']/df['od550aer']

dfe['ERF/AOD'] = (dfe['ERFt'] + dfe['od550aer'])/2

df=df.replace([np.inf, -np.inf], np.nan)
dfe = dfe.replace([np.inf, -np.inf], np.nan)
multimodel_mean =  df.mean(axis=0)

std = df.std(axis=0)
dferror = std/np.sqrt(len(df.index))



df.loc['Multi-model',:] = multimodel_mean
dfe.loc['Multi-model',:] = dferror

In [8]:
def greater_than_variability(v,dfe, std_keys=[],alpha=0.01):
    if v.name in std_keys:
        cond = np.abs(v) > 2*dfe.loc[:,v.name]
    else:
        cond = dfe.loc[:,v.name]<=alpha

       
    df = pd.Series(np.where(cond,'font-weight: bold', None),index=v.index,name=v.name)
    cond = np.abs(v['Multi-model'])>dfe.loc['Multi-model', v.name]
    if cond:
        df['Multi-model'] = 'color: blue; font-weight: bold'
    else:
        df['Multi-model']=None
    return df

In [9]:
forcing_translation = {
    'SWDirectEff': 'SW Fari',
    'LWDirectEff': 'LW Fari',
    'DirectEff': 'Fari',
    'CloudEff': 'Faci',
    'SWCloudEff':'SW Faci',
    'ERFt':'ERF total',
    'LWCloudEff': 'LW Faci',
    'ERFtcsaf': 'Albedo forcing',
    'feedback_tot_alpha' : '&alpha; emissions Wm-2 K-1'
}

In [10]:
forcing_componets =['SWDirectEff','LWDirectEff','DirectEff','SWCloudEff','LWCloudEff','CloudEff','ERFtcsaf','ERFt']
dfe_forcing = dfe[forcing_componets].rename(columns=forcing_translation)
df_forcing = df[forcing_componets].rename(columns=forcing_translation)
f = partial(greater_than_variability,dfe=dfe_forcing)


dst=df_forcing.style.apply(f, axis=0)\
.bar(color=['#2717a3','#d95148'], height=50, align='zero',
              width=60, props="width: 100px; border-right: 1px solid black;", axis=0)\
    .format(precision=3, na_rep='')\
    .set_caption('Decomposition of dust radiatve radiative forcings W m-2') .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: bottom; font-size:1.5em;'
 }], overwrite=False)
dst

In [11]:
dfi.export(dst,snakemake.output.forcing_table, dpi=300)


In [12]:
diag_translation = {
    'clivi' : 'Ice Water Path g m-2',
    'clt'   : 'Cloud cover (%)',
    'clwvi'   : 'Liquid Water Path g m-2',
    'pr'    : 'Precipitation g m-2 s-1'
    
}

In [21]:
diagnostics_compotents = ['clivi','clt','clwvi','pr']
dfe_diag = dfe[diagnostics_compotents].rename(columns=diag_translation)
df_diag = df[diagnostics_compotents].rename(columns=diag_translation)

f = partial(greater_than_variability,dfe=dfe_diag)


dst=df_diag.style.apply(f, axis=0)\
.bar(color=['#2717a3','#d95148'], height=50, align='zero',
              width=60, props="width: 100px; border-right: 1px solid black;", axis=0)\
    .format(precision=3, na_rep='', 
            formatter={
                'Precipitation g m-2 s-1': lambda x: "{:.2e}".format(x*1e3),
                'Liquid Water Path g m-2': lambda x: "{:.3f}".format(x*1e3),
                'Ice Water Path g m-2': lambda x: "{:.3f}".format(x*1e3)
            }
           )\
    .set_caption('Absolute change between control and 2x-dust') .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: bottom; font-size:1.5em;'
 }], overwrite=False)
dst

In [14]:
dfi.export(dst,snakemake.output.diagnostics_table_path, dpi=300)

In [15]:
df.columns

In [22]:
optics_translation = {
   'concdust' :  '&Delta; Dust burden Tg', 
#     'atmabs': 'Atmoshperic absorption W m-2',
    'od550aer' : '&Delta; AOD 550mn',
    'abs550aer' : '&Delta; AAOD 550mn',
    'delta_MEE_dustMEE' : '&Delta; MEE g m-2',
    'delta_MEEabs_dustMEEabs' : '&Delta; MEE absorption g m-2',
#     'ERF/AOD'  : 'ERFt per AOD change W m-2',
    'emidust'  : '&Delta; Dust emissions Tg year-1',
    'dulifetime_dulifetime' : 'Lifetime (Days)'
    
}

diagnostics_compotents = ['emidust','concdust','dulifetime_dulifetime','od550aer','abs550aer',
                          'delta_MEE_dustMEE','delta_MEEabs_dustMEEabs']#'atmabs','ERF/AOD']
dfe_diag = dfe[diagnostics_compotents].rename(columns=optics_translation)
df_diag = df[diagnostics_compotents].rename(columns=optics_translation)

f = partial(greater_than_variability,dfe=dfe_diag,std_keys=['Lifetime (Days)','&Delta; MEE absorption g m-2',
                                                           '&Delta; MEE g m-2'])


dst=df_diag.style.apply(f, axis=0)\
.bar(color=['#2717a3','#d95148'], height=50,
              width=60, props="width: 100px; border-right: 1px solid black;", axis=0)\
    .format(precision=3, na_rep='', 
            formatter={
#                 'Atmoshperic absorption W m-2': lambda x: "{:.2f}".format(x),
                '&Delta; AOD 550mn': lambda x: "{:.3f}".format(x),
                '&Delta; AAOD 550mn': lambda x: "{:.1E}".format(x),
                'ERFt per AOD change': lambda x: "{:.2f}".format(x),
                '&Delta; Dust emissions Tg year-1' : lambda x: "{:.2f}".format(x),
                'Lifetime (Days)' : lambda x: "{:.1f}".format(x),
                '&Delta; Dust burden Tg' : lambda x: "{:.1f}".format(x)
            }
           )\
    .set_caption('Absolute change between control and 2x-dust') .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: bottom; font-size:1.5em;'
 }], overwrite=False)



dst


In [23]:
dfi.export(dst,snakemake.output.optics_diag_table, dpi=300)

In [24]:
feedback_translation = {
    'feedback_tot_alpha' : '&alpha; emissions Wm-2 K-1',
    'feedback_tot_dCdT'  : '&Delta; emission /&Delta;T Tgyr-1 K-1' 
}

In [26]:

feedback_components = ['feedback_tot_dCdT','feedback_tot_alpha']
dfe_diag = dfe[feedback_components].rename(columns=feedback_translation)
df_diag = df[feedback_components].rename(columns=feedback_translation)
df_diag=df_diag.drop('IPSL-CM6A-LR-INCA', axis=0)
dfe_diag=dfe_diag.drop('IPSL-CM6A-LR-INCA', axis=0)
f = partial(greater_than_variability,dfe=dfe_diag, std_keys=['&Delta; emission /&Delta;T Tgyr-1 K-1',
                                                            '&alpha; emissions Wm-2 K-1'])
# df_diag=df_diag.drop('IPSL-CMA6-LR-INCA', axis=1)

dst=df_diag.style.apply(f, axis=0)\
.bar(color=['#2717a3','#d95148'], height=50, align='zero',
              width=60, props="width: 100px; border-right: 1px solid black;", axis=0)\
    .format(precision=3, na_rep='', formatter={
    '&Delta; emission /&Delta;T Tgyr-1 K-1': lambda x: "{:.1f}".format(x),
}
           )\
    .set_caption('Emission emission change per temperature change and Feedback parameter') .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: bottom; font-size:1.2em;'
 }], overwrite=False)
dst

In [27]:
dfi.export(dst,snakemake.output.feedback_table, dpi=300)

In [28]:
if snakemake.output.outpath.endswith('csv'):
    df.to_csv(snakemake.output.outpath)
elif snakemake.output.outpath.endswith('tex'):
    col_format = ['m{5em}'] + ['m{1.5cm}' for i in range(len(df.columns))]
    col_format = ''.join(col_format)
    with open(snakemake.output.outpath, 'w') as f:
        df.to_latex(buf=f,  na_rep='', col_space=3, longtable=False, column_format= col_format)